In [22]:
import time, re
from collections import Counter
import numpy as np

Mycket av koden är hämtat från https://github.com/udacity/deep-learning/tree/master/embeddings 

In [4]:
PATH = 'data/'

In [5]:
%%time
with open(f'{PATH}ahnflefn.txt') as f:
    text = f.read()

CPU times: user 2.19 ms, sys: 1.68 ms, total: 3.87 ms
Wall time: 3.92 ms


In [37]:
# Begränsa textstorleken
text = text[:1000000]

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [6]:
def preprocess(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('-\n', '')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

In [7]:
%%time 
#CPU times: user 48.3 s, sys: 7.27 s, total: 55.6 s
words = preprocess(text)
print(words[:30])

['von', '<td', 'af', 'honom', '<td', '<td', 'efter', 'tryckta', 'och', '<td', '<td', '<PERIOD>', '<td', '<td', 'l', '<PERIOD>', '<PERIOD>', '<td', '<td', '<COLON>', '2', '<PERIOD>', '<PERIOD>', 'sista', '<EXCLAMATION_MARK>', '</i>', 'nyligen', 'på', 'l', '<PERIOD>']
CPU times: user 27.4 ms, sys: 5.39 ms, total: 32.8 ms
Wall time: 32.4 ms


In [8]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 40864
Unique words: 1029


## Make chunks

In [7]:
len(words)

40864

In [8]:
def make_chunks (words, chunk_size):
    num_chunks = len(words)//chunk_size
    return [x * chunk_size for x in range (num_chunks)]
    

## Kapitel

In [9]:
def make_freq (words):
    counter = Counter(words)
    sum_of = sum(counter.values())
    return {x:(counter[x]/sum_of) for x in counter}

In [10]:
def compare_freqs (sub_dict, total_dict):
    return {word:(sub_dict[word]/total_dict[word]) for word in sub_dict}
    

## Kapitel

In [11]:
def characteristic_words(words, chunk_size):
    freq_total = make_freq(words)
    start_at = make_chunks(words, chunk_size)
    for x in start_at:
        words_sub = words[x: x+chunk_size]
        freq_sub = make_freq(words_sub)
        comp_freqs = compare_freqs(freq_sub, freq_total)
        list1 = sorted(comp_freqs, key=comp_freqs.get, reverse=True)
        print (list1[:10], '\n')
    return comp_freqs
        

In [30]:
# Korta ner texten?
words = words[:1000000]

In [12]:
%%time
x = characteristic_words(words, 5000)

['rothman', 'stobæi', '</b>', 'stobæus', 'utgifvaren', '<tab>', 'celsius', 'fadern', 'gossen', 'lappland'] 

['gulden', 'burmann', 'dillenius', 'harderwijk', 'återvända', 'amsterdam', 'boerhave', 'ni', 'er', 'fallet'] 

['horn', '-', '1739', 'royen', 'förnuft', 'lärt', '1738', 'krafter', 'linnæi', 'tjena'] 

['sen', 'blåkulla', 'ägg', 'träden', 'bergen', 'vestgötaresan', 'maskar', '<i>ur', 'fisk', '1746'] 

['betyder', 'bänk', 'högsätet', 'bruden', 'brudgummen', 'gafveln', 'rutan', 'bordet', 'dörren', 'sängen'] 

['kommission', 'blackwell', 'olyckan', 'gyldensten', 'stobè', 'bl', 'ständerna', 'g', 'sättes', 'xii'] 

['clerck', 'kjärrman', '<tab><tab><tab><tab><i>upsala', 'b', 'br', '1772', 'låtit', 'hafver', 'hustru', 'himmel'] 

['herbariet', 'of', 'sir', 'society', 'the', 'linnean', 'förtroende', 'född', 'smith', 'linnéska'] 

CPU times: user 10.8 ms, sys: 581 µs, total: 11.4 ms
Wall time: 11 ms


In [13]:
list1 = sorted(x, key=x.get, reverse=True)
x[list1[0]]

8.1728

In [14]:
granska_ord = Counter(words)

In [15]:
granska_ord['träden']

6

Men hur får man fram när man byter ämne i texten

## Bigrams

In [16]:
# https://stackoverflow.com/questions/43473736/most-common-2-grams-using-python

def bigrams(words):
    bigr = zip(words, words[1:])
    #print (list(bigr))
    counts = Counter(bigr)
    #print(counts)
    print(counts.most_common()[:10])

In [18]:
# https://stackoverflow.com/questions/43473736/most-common-2-grams-using-python

def trigrams(words):
    trigr = zip(words, words[1:], words[2:])
    #print (list(bigr))
    counts = Counter(trigr)
    #print(counts)
    print(counts.most_common()[:10])

In [68]:
bigrams('som om man inte om man skulle'.split())

[(('om', 'man'), 2), (('som', 'om'), 1), (('man', 'inte'), 1), (('inte', 'om'), 1), (('man', 'skulle'), 1)]


In [17]:
bigrams(words)

[(('<COMMA>', 'som'), 466), (('<COMMA>', 'och'), 362), (('<COMMA>', 'att'), 330), (('<COMMA>', '<COMMA>'), 224), (('<COMMA>', 'men'), 190), (('och', '<COMMA>'), 138), (('en', '<COMMA>'), 120), (('<COMMA>', 'så'), 106), (('<COMMA>', 'i'), 99), (('i', '<COMMA>'), 99)]


In [19]:
trigrams(words)

[(('<COMMA>', '<COMMA>', '<COMMA>'), 95), (('<COMMA>', 'att', 'han'), 38), (('<COMMA>', 'och', '<COMMA>'), 31), (('<LEFT_PAREN>', '<RIGHT_PAREN>', '<COMMA>'), 29), (('den', '<COMMA>', 'som'), 26), (('<COMMA>', '<COMMA>', 'och'), 26), (('<COMMA>', 'så', 'att'), 23), (('<COMMA>', 'och', 'då'), 22), (('en', '<COMMA>', 'som'), 22), (('<COMMA>', 'som', 'i'), 21)]


In [25]:
re.sub('<\w*>|<\/\w*>','',text)

'CARL von LINNÈS\n\nLEFNADSMINNEN\n<table o>\n<td c>tecknade af honom sjelf\n\n<td c>*\n\n<td c>med tillägg efter tryckta och otryckta källor\n<td c>af\n<td c>ARVID AHNFELT.\n\n<td c>*\n\n<td c>stockholm\n\noscar l. lamms förlag.\n\n<td c>*\n\n<td c>Pris: 2 Kr. 50 öre.\n\n\nObs omslagets sista sida!\n\n\nIntressanta Nyheter\n\nNyligen utkomna på Oscar L. Lamms förlag\n\nInteriörer ur det literära Stockholmslifvet\n\nunder förra hälften af vårt århundrade.\n\nSamlade och belysta af\n\nARVID AHNFELT.\n\nI.\n\n2 kr. 50 öre.\n\nObs! — — A. har lemnat några högt intressanta bidrag, som står\noss så nära, men som likväl för flertalet är långt mera obekant, än man\nkunde tro — —\n\n— — Med den lätta stil, som alltid utmärker A:s penna, tecknar han\ndessa bilder. — —Boken rekommenderas synnerligen.\n(T. f. V.  st. o. L.)\n\n— — Det är med verkligt nöje vi läst detta häfte — — den lätthet,\nmed hvilken förf. behandlar sitt ämne, bidrager i sin mån att göra \ndetsamma intressant.(C. T.)\n\n—\n\n

In [24]:
words.type()

AttributeError: 'list' object has no attribute 'type'

In [9]:
# Från https://github.com/udacity/deep-learning/tree/master/embeddings 
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab, word_counts

In [25]:
vocab_to_int, int_to_vocab, word_counts = create_lookup_tables(words)
int_words = np.array([vocab_to_int[word] for word in words])

In [26]:
int_words

array([648, 209,   5, ...,  19, 660,   3])

In [27]:
vocab_to_int['nöje']

279

In [43]:
occs = np.where(int_words == vocab_to_int['fisk'])

In [31]:
reading_context = 300

In [34]:
sample = words [18515-20:18515+300]

In [35]:
text = ' '.join(sample); text

'vestgötaresan <PERIOD> </i> marstrand den juli 1746 <PERIOD> kallades här i den <COMMA> som icke sjelf kan ned i att fisk <COMMA> utan endast är till bland <PERIOD> man med nöje huru denne de andra <COMMA> så snart de fått någon fisk <COMMA> och ej igen att dem måste ut den fisk han och redan <PERIOD> jag har sett med förundran på en <COMMA> som jag flere år haft i <COMMA> att om han fått aldrig så litet mat och någon sedermera litet efter honom <COMMA> har han då strax det han <PERIOD> denna att lätt har skaparen användt att vår <COMMA> ty som ofta mera än de böra <COMMA> så hafva de också väl råd att gifva åt <COMMA> men deremot har naturen likväl så <COMMA> att denne ej allt för mycket får sig <COMMA> hvarför han ock är den af alla <PERIOD> härtill kommer att denne icke är mycket <COMMA> ty under tiden måste <COMMA> då de ej hafva något på <COMMA> och kasta för honom mat <COMMA> hvilken han ock håller till <PERIOD> är mycket <COMMA> så att han alltid tager i luften <COMMA> då den å

In [44]:
occs = occs[0]
occs

array([14252, 15780, 18515, 18536, 18546, 18707, 33989])

In [41]:
# https://stackoverflow.com/questions/2400840/finding-differences-between-elements-of-a-list
dists = [j-i for i, j in zip(occs[:-1], occs[1:])]
dists

[1528, 2735, 21, 10, 161, 15282]